# # EDA And Baseline on Motor Temperature Estimation With H2O


Dependências básicas

In [1]:
!pip install numpy
!pip install pycaret

You should consider upgrading via the '/home/pedro/meu_projeto/meu_projeto_env/bin/python -m pip install --upgrade pip' command.


You should consider upgrading via the '/home/pedro/meu_projeto/meu_projeto_env/bin/python -m pip install --upgrade pip' command.


Importando Bibliotecas

In [2]:
import numpy as np #Biblioteca "matemática"
import pandas as pd #Biblioteca para manipulação e análise de dados
import matplotlib.pyplot as plt #Extenção da biblioteca que faz a pltagem de gráficos e pontos
import seaborn as sns
import os #Funcionalidade simplificadas de sistema operacionais
print(os.listdir())

['Book - Hands-on atuomated machine learning.ipynb', 'pandas_profiling', 'Curso Data science.ipynb', 'jupyter-nbconvert', 'python', 'databricks', 'archive.zip', 'numba', 'AutoSklearn - Eletric Motor.ipynb', 'jupyter-bundlerextension', 'wheel-3.8', 'melb_data.csv', 'mako-render', 'jupyter-troubleshoot', 'Eletric Motor_Autosklearn_H2O.ipynb', 'activate.csh', 'f2py3', 'cython', 'futurize', 'jsonschema', 'htmlmin', 'pip', 'jupyter-qtconsole', 'fonttools', 'alembic', 'wordcloud_cli', 'pip3', 'dbfs', 'activate_this.py', 'mlruns', 'jupyter-serverextension', 'pyftsubset', 'ipython', 'jupyter-console', 'f2py3.8', 'send2trash', 'distro', 'dask-scheduler', 'pip-3.8', 'python3', 'wheel3.8', 'pip3.8', 'phik_trial', 'ttx', 'gunicorn', 'jupyter-run', 'activate.nu', 'f2py', 'ipython3', 'pygmentize', 'jupyter-kernelspec', 'spacy', 'datahr.csv', 'flask', 'jupyter-migrate', 'find_similar_images.py', 'Eletric Motor_Autosklearn.ipynb', 'jupyter-execute', 'wheel', 'jupyter-nbextension', '.ipynb_checkpoints'

Leitura e tratamento do dataset

In [3]:
df=pd.read_csv('measures_v2.csv', usecols=[0,1,2,3,4,5,6,7,8,9,10,11])
target = df.pop('pm') #Temperatura do rotor
df = pd.concat([df, target], axis=1)
df = df.sample(frac=1,random_state=0) #embaralha os dados do dataframe #Ajuda a previnir o overfitting
df.reset_index(drop=True, inplace=True) #Faz com que o Index volte a ser o que era antes
df


,u_q,coolant,stator_winding,u_d,stator_tooth,motor_speed,i_d,i_q,stator_yoke,ambient,torque,pm
0,41.938923,18.744030,66.684830,-123.478027,46.080647,4749.964355,-187.964111,77.635490,33.169899,20.551407,7.486735e+01,40.030872
1,-0.431508,59.902590,85.079312,-0.878644,76.299257,0.057160,-2.000745,1.097449,67.216376,24.467134,-2.614878e-06,72.503215
2,-1.541598,33.149664,48.669293,-0.333442,45.330586,0.001482,-2.000673,1.097664,39.476950,23.169348,5.436636e-14,43.026863
3,42.387482,44.949261,104.791174,-123.337533,90.274398,5112.368164,-181.587703,68.767176,72.123242,27.374628,6.631759e+01,74.825417
4,15.335679,18.755226,113.366333,-130.067474,84.144737,3999.963135,-205.157623,97.281723,57.477020,23.940556,9.412620e+01,89.903763
...,...,...,...,...,...,...,...,...,...,...,...,...
1330811,12.093378,18.362038,19.795088,0.766441,19.273512,249.997833,-1.999840,1.097852,19.066999,22.962929,1.049427e-01,25.814903
1330812,23.644573,18.671892,39.454746,-22.069843,32.786079,499.995819,-43.512096,132.618576,27.362280,23.022329,1.033085e+02,35.359821
1330813,31.839993,54.416758,102.315358,56.278286,88.524355,1096.309584,-62.842545,-174.414836,75.441479,27.976668,-1.377123e+02,82.917509
1330814,94.615028,18.818180,36.839790,-1.240177,36.204067,1999.979004,-1.999915,1.098047,30.371397,23.878441,-4.387210e-01,68.415085


Separação dos dados de treino e dados de teste

In [4]:
split_index=int(len(df) * 0.75)

train_df = df[:split_index] #Primeiros 75%
test_df = df[split_index:] #outros 25% restantes

train_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998112 entries, 0 to 998111
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   u_q             998112 non-null  float64
 1   coolant         998112 non-null  float64
 2   stator_winding  998112 non-null  float64
 3   u_d             998112 non-null  float64
 4   stator_tooth    998112 non-null  float64
 5   motor_speed     998112 non-null  float64
 6   i_d             998112 non-null  float64
 7   i_q             998112 non-null  float64
 8   stator_yoke     998112 non-null  float64
 9   ambient         998112 non-null  float64
 10  torque          998112 non-null  float64
 11  pm              998112 non-null  float64
dtypes: float64(12)
memory usage: 91.4 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332704 entries, 998112 to 1330815
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   --

Separação da última linha em dados de treino e dados de teste

In [6]:
X_train = train_df.to_numpy()[:, :-1] 
y_train = train_df.to_numpy()[:, -1]

X_test = test_df.to_numpy()[:, :-1]
y_test = test_df.to_numpy()[:, -1] 


Seguindo instalação segundo diretrizes do pycaret para treinamento

In [7]:
from pycaret.regression import *

S = setup(train_df, target = 'pm')

,Description,Value
0,session_id,4559
1,Target,pm
2,Original Data,"(998112, 12)"
3,Missing Values,False
4,Numeric Features,11
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(698678, 10)"


Treinamento

In [ ]:
best_model = compare_models() 

IntProgress(value=0, description='Processing: ', max=94)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,18:50:20
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Regressor


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
dt,Decision Tree Regressor,0.1802,1.0364,1.0174,0.9971,0.0184,0.0033,7.8840
knn,K Neighbors Regressor,1.7005,15.7262,3.9655,0.9565,0.0688,0.0293,5.6360
lr,Linear Regression,5.4280,52.4559,7.2426,0.8549,0.1432,0.1049,1.3660
ridge,Ridge Regression,5.4280,52.4559,7.2426,0.8549,0.1432,0.1049,0.1060
lar,Least Angle Regression,5.4281,52.4560,7.2426,0.8549,0.1432,0.1049,0.1350
br,Bayesian Ridge,5.4280,52.4559,7.2426,0.8549,0.1432,0.1049,0.6240
lasso,Lasso Regression,5.5506,55.7496,7.4665,0.8458,0.1468,0.1080,9.6680
en,Elastic Net,5.5706,56.2111,7.4974,0.8445,0.1473,0.1086,9.2080
huber,Huber Regressor,6.1811,71.5680,8.4593,0.8020,0.1650,0.1191,6.8720
omp,Orthogonal Matching Pursuit,8.0105,111.1480,10.5426,0.6926,0.1948,0.1560,0.1150


In [24]:
print(best_model)

[]


Transformando o Frame para o formato da biblioteca H2O


Identificando os valores das características e o que é necessário predizer